---
title: "Image Retrieval"
date: today
date-format: long
author: "Steven  Ndung'u et al."
format:
  html:
    toc: false
    toc-depth: 3
    toc-location: left
    page-layout: full
    theme:
          light: flatly
          dark: darkly
    number-sections: false
    highlighting: true
    smooth-scroll: true
    code-fold: true
    highlighting-style: GitHub
    self-contained: true
execute:
    echo: true
    warning: false
    enable: true

title-block-banner: true

---

```{=html}
<style type="text/css">

h1.title {
  font-size: 0px;
  color: White;
  text-align: center;
}
h4.author { /* Header 4 - and the author and data headers use this too  */
    font-size: 16px;
  font-family: "Source Sans Pro Semibold", Times, serif;
  color: Red;
  text-align: center;
}
h4.date { /* Header 4 - and the author and data headers use this too  */
  font-size: 16px;
  font-family: "Source Sans Pro Semibold", Times, serif;
  color: Red;
  text-align: center;
}
</style>
```


------------------------------------------------------------------------
:::{.column-page}

::: {style="text-align:center"}
<h2>Model Evaluation COSFIRE Filters Approach</h2>
:::

</br>

# Introduction

We obtain the 26 statistically significant sets of hyperparameters from the classification paper along with their respective training, validation, and test descriptors. Based on these descriptors, we perform image hashing for each set of descriptors using a selected set of MLP hyperparameters (for the grid search). 


In [ ]:
#| echo: false
#| code-fold: false
#| 
###################################################

###################################################
#$Env:QUARTO_PYTHON = "C:\Users\P307791\Anaconda3\python.exe"
import os
os.environ['PYTHONHASHSEED'] = 'python'
from scipy import stats

from IPython.display import display, Markdown, HTML
from itables import init_notebook_mode
init_notebook_mode(all_interactive=True)
from itables import show

import torch.nn as nn

import plotly.express as px
import pandas as pd
import plotly.graph_objects as go
import plotly.io as pio
#pio.renderers.default = "notebook"

import pandas as pd
import numpy as np
import re
from scipy import stats

import seaborn as sns
import matplotlib.pyplot as plt


def find_max(row):
    return row.max()


def colsum(x):
    return sum(np.isnan(x))


# l1_reg_values = [0,0.0001, 0.00001, 1e-8]
# l2_reg_values = [0,0.0001, 0.00001, 1e-8]

# l1_reg_values = [0,1e-8]
# l2_reg_values = [0,1e-8]

# query = (
#     "output_size == @output_size and "
#     "l1_reg in @l1_reg_values and "
#     "l2_reg in @l2_reg_values"
# )


The outputs below represent an example of a single experiment based on an array of hyperparameters considered to determine the optimal configuration of the MLP hashing architecture (For bit size 32).

::: {.callout-tip}

tiply, each row represents a unique combination of MLP hyperparameters, and every column represents the results yielded by each of the 26 statistically significant sets of descriptors. 

:::

</br>

::: {.panel-tabset}


In these experiments, we have considered four different bit sizes.


##  Bit size 48


The results in this presentation are from two experimental designs:




The thresholding is based on fixed values between -1 and 1 on a step size of 0.1.



::: {.panel-tabset}

#### Validation Data mAP Results Preview:


In [ ]:
#%%
layer_vsn = 'v3_layers'

dt_valid  = pd.read_csv(f"final paper results/merged_validation_runs_wide_format_abs_24082024_{layer_vsn}.csv")


dt_test  = pd.read_csv(f"final paper results/merged_test_runs_wide_format_abs_24082024_{layer_vsn}.csv")



output_size = 48

query = (
    "output_size == @output_size "
)

dt_valid = dt_valid.query(query)

dt_test = dt_test.query(query)

In [ ]:
#| echo: false
#| code-fold: false
#| 
html_table = dt_valid.to_html(index=True)

# Wrap in a scrollable div
scrollable_table = f"""
<div style="height: 400px; width: 100%; overflow-x: auto; overflow-y: auto;">
    {html_table}
</div>
"""
# Display the scrollable table
display(HTML(scrollable_table))

</br>

#### Test Data mAP Results Preview:


In [ ]:
#| echo: false
#| code-fold: false
#| 
html_table = dt_test.to_html(index=True)

# Wrap in a scrollable div
scrollable_table = f"""
<div style="height: 400px; width: 100%; overflow-x: auto; overflow-y: auto;">
    {html_table}
</div>
"""
# Display the scrollable table
display(HTML(scrollable_table))

:::

</br>

The global mean maximum row:


In [ ]:
#| echo: false
#| code-fold: false

################
dt_valid_sub = dt_valid[list(dt_valid.columns[dt_valid.columns.str.startswith('mAP_valid')])]

# Apply the function row-wise
dt_valid_sub['average_map'] = dt_valid_sub.apply(np.mean, axis=1)
dt_valid_sub.sort_values(by='average_map', ascending=False, inplace=True)

#maximum mean value
dt_valid_sub['average_map'].max()



#########


# extract max value row.

max_index = dt_valid_sub['average_map'].idxmax()
max_row = dt_valid_sub.loc[max_index]
#print(max_row)
dt_valid_sub_mw = dt_valid_sub.copy()

In [ ]:
#| echo: false
#| code-fold: false
dd = dt_valid_sub.head(10)

html_table = dd.to_html(index=True)

# Wrap in a scrollable div
scrollable_table = f"""
<div style="height: 400px; width: 100%; overflow-x: auto; overflow-y: auto;">
    {html_table}
</div>
"""
# Display the scrollable table
display(HTML(scrollable_table))

#### Model performance on validation data (Mann-Whitney U Test)

##### Statistical Significance:


Using the global mean maximum row as the reference, we perform the right-tailed t-test to identify significant hyperparameters.


In [ ]:
pvalues_mw = []
pvalues_real_mw = []

for _,index in enumerate(dt_valid_sub.index):

  _, p_value_mw = stats.mannwhitneyu(max_row[:-1], dt_valid_sub.loc[index][:-1], alternative='greater')#, method = 'asymptotic')
  pvalues_real_mw.append(p_value_mw)
  pvalues_mw.append((p_value_mw >= 0.05)*1)
  

dt_valid_sub_mw['pvalues_mw'] = pvalues_real_mw
dt_valid_sub_mw['sig_mw'] = pvalues_mw

dt_valid_sub_mw1 = dt_valid_sub_mw.query('sig_mw == 1')


max_indexx = dt_valid_sub_mw1.iloc[:, :-3].mean().idxmax()

#print(sum(pvalues),max_indexx)

In [ ]:
#| echo: false
#| code-fold: false

dt_mw = dt_valid.loc[dt_valid_sub_mw1.index].sort_values(by = ['input_size', 'output_size', 'learning_rate', 'batch_size', 'alpha', 'margin', 'l1_reg', 'l2_reg'])
html_table = dt_mw.to_html(index=True)


# Wrap in a scrollable div
scrollable_table = f"""
<div style="height: 400px; width: 100%; overflow-x: auto; overflow-y: auto;">
    {html_table}
</div>
"""
# Display the scrollable table
display(HTML(scrollable_table))

print('Size of the All data: ',dt_valid_sub_mw.shape)

print('Size of the Sig data: ',dt_mw.shape)

In [ ]:
####################################################
# Use the hyperparameters to the test data.
####################################################
optimal_params_mw1 = dt_valid.loc[dt_valid_sub_mw1.index][['input_size', 'output_size', 'learning_rate', 'batch_size', 'alpha', 'margin', 'l1_reg', 'l2_reg']]
optimal_params_mw = optimal_params_mw1.reset_index()
optimal_params_mw.drop(['index'], axis=1, inplace = True)

#rotation invariance test data with optimal parameters
test_data_mw1 = pd.merge(optimal_params_mw, dt_test, on=['input_size', 'output_size', 'learning_rate', 'batch_size', 'alpha', 'margin', 'l1_reg', 'l2_reg'])
test_data_1_sub_mw = test_data_mw1[list(test_data_mw1.columns[test_data_mw1.columns.str.startswith('mAP_test')])]

Equivalent test data


In [ ]:
#| echo: false
#| code-fold: false
test_data_mw1 = test_data_mw1.sort_values(by = ['input_size', 'output_size', 'learning_rate', 'batch_size', 'alpha', 'margin', 'l1_reg', 'l2_reg'])
html_table = test_data_mw1.to_html(index=True)

# Wrap in a scrollable div
scrollable_table = f"""
<div style="height: 400px; width: 100%; overflow-x: auto; overflow-y: auto;">
    {html_table}
</div>
"""
# Display the scrollable table
display(HTML(scrollable_table))
print('Size of the test data: ',test_data_mw1.shape)

::: {.panel-tabset}

##### Model performance on valid data 

Average & Std Deviation of the Significant rows:


In [ ]:
#| echo: false
#| code-fold: false
res = dt_valid_sub_mw1.iloc[:,:-3].describe().iloc[1:3]

res_valid_mwu_48 = res.T.sort_values(by=['mean'], ascending= False)

html_table = res_valid_mwu_48.to_html(index=True)

# Wrap in a scrollable div
scrollable_table = f"""
<div style="height: 400px; width: 100%; overflow-x: auto; overflow-y: auto;">
    {html_table}
</div>
"""
# Display the scrollable table
display(HTML(scrollable_table))


##### Model performance on Test data 
</br>
We then apply the hyperparameters to the test set and average the results.


In [ ]:
####################################################
# Use the hyperparameters to the test data.
####################################################
optimal_params = dt_valid.loc[dt_valid_sub_mw1.index][['input_size', 'output_size', 'learning_rate', 'batch_size', 'alpha', 'margin', 'l1_reg', 'l2_reg']].reset_index()

optimal_params.drop(['index'], axis=1, inplace = True)

#rotation invariance test data with optimal parameters
test_data_1 = pd.merge(optimal_params, dt_test, how='left')
test_data_1_sub = test_data_1[list(test_data_1.columns[test_data_1.columns.str.startswith('mAP_test')])]
test_data_1['average_map'] = test_data_1_sub.apply(np.mean, axis=1)

test_data_1.sort_values(by=['average_map'], ascending= False,inplace=True)

In [ ]:
#| echo: false
#| code-fold: false
res = test_data_1.describe().iloc[1:3][list(test_data_1.columns[test_data_1.columns.str.startswith('mAP_test')]) + ['average_map']]

res_test_mwu_48 = res.T.sort_values(by=['mean'], ascending= False)

html_table = res_test_mwu_48.to_html(index=True)

# Wrap in a scrollable div
scrollable_table = f"""
<div style="height: 400px; width: 100%; overflow-x: auto; overflow-y: auto;">
    {html_table}
</div>
"""
# Display the scrollable table
display(HTML(scrollable_table))

:::


In [ ]:
#| layout-ncol: 2
#| 
def extract_number(text):
    if isinstance(text, str):
        matches = re.findall(r'\d+', text)
        return int(matches[0]) if matches else 1
    return 1


res_valid_mwu_48['id'] = res_valid_mwu_48.index.to_series().apply(extract_number)
res_test_mwu_48['id'] = res_test_mwu_48.index.to_series().apply(extract_number)

res_comb = pd.concat([res_valid_mwu_48,res_test_mwu_48])
index_series = res_comb.index.to_series()

res_comb['type'] = np.where(
    index_series.str.contains('valid', case=False), 'valid',
    np.where(index_series.str.contains('test', case=False), 'test', 'unknown')
)
res_comb = res_comb.query("type !='unknown'")

res_comb =  res_comb.reset_index(drop=True)


res_test_mwu_48 = res_test_mwu_48.sort_values(by=['id']).reset_index().query("index !='average_map'")


res_valid_mwu_48 =  res_valid_mwu_48.reset_index(drop=True)
res_valid_mwu_48.columns = ['valid_score', 'std', 'id']
res_valid_mwu_48 = res_valid_mwu_48[['valid_score', 'id']]

res_test_mwu_48.columns = ['label2', 'test_score', 'std', 'id']
res_test_mwu_48 = res_test_mwu_48[['test_score', 'id']]


result_merged_mwu_48 = pd.merge(res_valid_mwu_48,res_test_mwu_48,on='id')


result_merged_mwu_48.sort_values(by=['valid_score'], ascending=False,inplace=True)

result_merged_mwu_48 = result_merged_mwu_48[['id','valid_score','test_score']]

html_table = result_merged_mwu_48.to_html(index=True)

# Wrap in a scrollable div
scrollable_table = f"""
<div style="height: 500px; width: 100%; overflow-x: auto; overflow-y: auto;">
    {html_table}
</div>
"""
# Display the scrollable table
display(HTML(scrollable_table))

#data_range2 = np.array(list(res_test_mwu_48['mean']) + list(res_valid_mwu_48['mean']))

# categories = [str(i) for i in range(1,26)]
# fig = go.Figure()

# # Valid
# fig.add_trace(go.Scatterpolar(
#        r=list(res_test_mwu_48['mean']),
#       theta=categories,
#       #fill='toself',
#       name='Test'
# ))

# # Test
# fig.add_trace(go.Scatterpolar(
#       r=list(res_valid_mwu_48['mean']),
#       theta=categories,
#       #fill='toself',
#       name='Valid'
# ))

# # Customization of chart
# fig.update_layout(
#   title={
#       'text': "",
#          'xanchor': 'center',
#       'yanchor': 'top'
#   },
#   polar=dict(
#     radialaxis=dict(
#       visible=True,
#       range=[88, 93]
#     )),
#   showlegend=True
# )

# fig.show()
# fig.write_image("MU_Test.png")

##  Bit size 32

::: {.panel-tabset}

#### Validation Data mAP Results Preview:


In [ ]:
#%%

dt_valid  = pd.read_csv(f"final paper results/merged_validation_runs_wide_format_abs_24082024_{layer_vsn}.csv")


dt_test  = pd.read_csv(f"final paper results/merged_test_runs_wide_format_abs_24082024_{layer_vsn}.csv")



output_size = 32

query = (
    "output_size == @output_size "
)

dt_valid = dt_valid.query(query)

dt_test = dt_test.query(query)

In [ ]:
#| echo: false
#| code-fold: false
#| 
html_table = dt_valid.to_html(index=True)

# Wrap in a scrollable div
scrollable_table = f"""
<div style="height: 400px; width: 100%; overflow-x: auto; overflow-y: auto;">
    {html_table}
</div>
"""
# Display the scrollable table
display(HTML(scrollable_table))

</br>

#### Test Data mAP Results Preview:


In [ ]:
#| echo: false
#| code-fold: false
#| 
html_table = dt_test.to_html(index=True)

# Wrap in a scrollable div
scrollable_table = f"""
<div style="height: 400px; width: 100%; overflow-x: auto; overflow-y: auto;">
    {html_table}
</div>
"""
# Display the scrollable table
display(HTML(scrollable_table))

:::

</br>

The global mean maximum row:


In [ ]:
#| echo: false
#| code-fold: false

################
dt_valid_sub = dt_valid[list(dt_valid.columns[dt_valid.columns.str.startswith('mAP_valid')])]

# Apply the function row-wise
dt_valid_sub['average_map'] = dt_valid_sub.apply(np.mean, axis=1)
dt_valid_sub.sort_values(by='average_map', ascending=False, inplace=True)

#maximum mean value
dt_valid_sub['average_map'].max()



#########


# extract max value row.

max_index = dt_valid_sub['average_map'].idxmax()
max_row = dt_valid_sub.loc[max_index]
#print(max_row)
dt_valid_sub_mw = dt_valid_sub.copy()

In [ ]:
#| echo: false
#| code-fold: false
dd = dt_valid_sub.head(10)

html_table = dd.to_html(index=True)

# Wrap in a scrollable div
scrollable_table = f"""
<div style="height: 400px; width: 100%; overflow-x: auto; overflow-y: auto;">
    {html_table}
</div>
"""
# Display the scrollable table
display(HTML(scrollable_table))

#### Model performance on validation data (Mann-Whitney U Test)

##### Statistical Significance:


Using the global mean maximum row as the reference, we perform the right-tailed t-test to identify significant hyperparameters.


In [ ]:
pvalues_mw = []
pvalues_real_mw = []

for _,index in enumerate(dt_valid_sub.index):

  _, p_value_mw = stats.mannwhitneyu(max_row[:-1], dt_valid_sub.loc[index][:-1], alternative='greater')#, method = 'asymptotic')
  pvalues_real_mw.append(p_value_mw)
  pvalues_mw.append((p_value_mw >= 0.05)*1)
  

dt_valid_sub_mw['pvalues_mw'] = pvalues_real_mw
dt_valid_sub_mw['sig_mw'] = pvalues_mw

dt_valid_sub_mw1 = dt_valid_sub_mw.query('sig_mw == 1')


max_indexx = dt_valid_sub_mw1.iloc[:, :-3].mean().idxmax()

#print(sum(pvalues),max_indexx)

In [ ]:
#| echo: false
#| code-fold: false




dt_mw = dt_valid.loc[dt_valid_sub_mw1.index].sort_values(by = ['input_size', 'output_size', 'learning_rate', 'batch_size', 'alpha', 'margin', 'l1_reg', 'l2_reg'])
html_table = dt_mw.to_html(index=True)


# Wrap in a scrollable div
scrollable_table = f"""
<div style="height: 400px; width: 100%; overflow-x: auto; overflow-y: auto;">
    {html_table}
</div>
"""
# Display the scrollable table
display(HTML(scrollable_table))

print('Size of the All data: ',dt_valid_sub_mw.shape)

print('Size of the Sig data: ',dt_mw.shape)

In [ ]:
dt_mw11 = dt_mw[list(dt_mw.columns[dt_mw.columns.str.startswith('mAP_valid')])]
# Apply the function row-wise
dt_mw11['max_map'] = dt_mw11.apply(np.max, axis=1)
dt_mw11 = dt_mw11.sort_values(by = ['max_map'],ascending=False)

html_table = dt_mw11.to_html(index=True)


# Wrap in a scrollable div
scrollable_table = f"""
<div style="height: 400px; width: 100%; overflow-x: auto; overflow-y: auto;">
    {html_table}
</div>
"""
# Display the scrollable table
display(HTML(scrollable_table))

In [ ]:
####################################################
# Use the hyperparameters to the test data.
####################################################
optimal_params_mw1 = dt_valid.loc[dt_valid_sub_mw1.index][['input_size', 'output_size', 'learning_rate', 'batch_size', 'alpha', 'margin', 'l1_reg', 'l2_reg']]
optimal_params_mw = optimal_params_mw1.reset_index()
optimal_params_mw.drop(['index'], axis=1, inplace = True)

#rotation invariance test data with optimal parameters
test_data_mw1 = pd.merge(optimal_params_mw, dt_test, on=['input_size', 'output_size', 'learning_rate', 'batch_size', 'alpha', 'margin', 'l1_reg', 'l2_reg'])
test_data_1_sub_mw = test_data_mw1[list(test_data_mw1.columns[test_data_mw1.columns.str.startswith('mAP_test')])]

Equivalent test data


In [ ]:
#| echo: false
#| code-fold: false
test_data_mw1 = test_data_mw1.sort_values(by = ['input_size', 'output_size', 'learning_rate', 'batch_size', 'alpha', 'margin', 'l1_reg', 'l2_reg'])
html_table = test_data_mw1.to_html(index=True)

# Wrap in a scrollable div
scrollable_table = f"""
<div style="height: 400px; width: 100%; overflow-x: auto; overflow-y: auto;">
    {html_table}
</div>
"""
# Display the scrollable table
display(HTML(scrollable_table))
print('Size of the test data: ',test_data_mw1.shape)

::: {.panel-tabset}

##### Model performance on valid data 

Average & Std Deviation of the Significant rows:


In [ ]:
#| echo: false
#| code-fold: false
res = dt_valid_sub_mw1.iloc[:,:-3].describe().iloc[1:3]

res_valid_mwu_32 = res.T.sort_values(by=['mean'], ascending= False)

html_table = res_valid_mwu_32.to_html(index=True)

# Wrap in a scrollable div
scrollable_table = f"""
<div style="height: 400px; width: 100%; overflow-x: auto; overflow-y: auto;">
    {html_table}
</div>
"""
# Display the scrollable table
display(HTML(scrollable_table))


##### Model performance on Test data 
</br>
We then apply the hyperparameters to the test set and average the results.


In [ ]:
####################################################
# Use the hyperparameters to the test data.
####################################################
optimal_params = dt_valid.loc[dt_valid_sub_mw1.index][['input_size', 'output_size', 'learning_rate', 'batch_size', 'alpha', 'margin', 'l1_reg', 'l2_reg']].reset_index()
optimal_params.drop(['index'], axis=1, inplace = True)

optimal_params.to_csv('optimal_params_32bit.csv',index=False)
#rotation invariance test data with optimal parameters
test_data_1 = pd.merge(optimal_params, dt_test, how='left')
test_data_1_sub = test_data_1[list(test_data_1.columns[test_data_1.columns.str.startswith('mAP_test')])]
test_data_1['average_map'] = test_data_1_sub.apply(np.mean, axis=1)

test_data_1.sort_values(by=['average_map'], ascending= False,inplace=True)

In [ ]:
#| echo: false
#| code-fold: false
res = test_data_1.describe().iloc[1:3][list(test_data_1.columns[test_data_1.columns.str.startswith('mAP_test')]) + ['average_map']]

res_test_mwu_32 = res.T.sort_values(by=['mean'], ascending= False)

html_table = res_test_mwu_32.to_html(index=True)

# Wrap in a scrollable div
scrollable_table = f"""
<div style="height: 400px; width: 100%; overflow-x: auto; overflow-y: auto;">
    {html_table}
</div>
"""
# Display the scrollable table
display(HTML(scrollable_table))

:::


In [ ]:
res_valid_mwu_32['id'] = res_valid_mwu_32.index.to_series().apply(extract_number)
res_test_mwu_32['id'] = res_test_mwu_32.index.to_series().apply(extract_number)

res_comb = pd.concat([res_valid_mwu_32,res_test_mwu_32])
index_series = res_comb.index.to_series()

res_comb['type'] = np.where(
    index_series.str.contains('valid', case=False), 'valid',
    np.where(index_series.str.contains('test', case=False), 'test', 'unknown')
)
res_comb = res_comb.query("type !='unknown'")

res_comb =  res_comb.reset_index(drop=True)


res_test_mwu_32 = res_test_mwu_32.sort_values(by=['id']).reset_index().query("index !='average_map'")


res_valid_mwu_32 =  res_valid_mwu_32.reset_index(drop=True)
res_valid_mwu_32.columns = ['valid_score', 'std', 'id']
res_valid_mwu_32 = res_valid_mwu_32[['valid_score', 'id']]

res_test_mwu_32.columns = ['label2', 'test_score', 'std', 'id']
res_test_mwu_32 = res_test_mwu_32[['test_score', 'id']]


result_merged_mwu_32 = pd.merge(res_valid_mwu_32,res_test_mwu_32,on='id')


result_merged_mwu_32.sort_values(by=['valid_score'], ascending=False,inplace=True)

result_merged_mwu_32 = result_merged_mwu_32[['id','valid_score','test_score']]

html_table = result_merged_mwu_32.to_html(index=True)

# Wrap in a scrollable div
scrollable_table = f"""
<div style="height: 500px; width: 100%; overflow-x: auto; overflow-y: auto;">
    {html_table}
</div>
"""
# Display the scrollable table
display(HTML(scrollable_table))

</br>

##  Bit size 24

::: {.panel-tabset}

#### Validation Data mAP Results Preview:


In [ ]:
#%%

dt_valid  = pd.read_csv(f"final paper results/merged_validation_runs_wide_format_abs_24082024_{layer_vsn}.csv")


dt_test  = pd.read_csv(f"final paper results/merged_test_runs_wide_format_abs_24082024_{layer_vsn}.csv")



output_size = 24

query = (
    "output_size == @output_size "
)

dt_valid = dt_valid.query(query)

dt_test = dt_test.query(query)

In [ ]:
#| echo: false
#| code-fold: false
#| 
html_table = dt_valid.to_html(index=True)

# Wrap in a scrollable div
scrollable_table = f"""
<div style="height: 400px; width: 100%; overflow-x: auto; overflow-y: auto;">
    {html_table}
</div>
"""
# Display the scrollable table
display(HTML(scrollable_table))

</br>

#### Test Data mAP Results Preview:


In [ ]:
#| echo: false
#| code-fold: false
#| 
html_table = dt_test.to_html(index=True)

# Wrap in a scrollable div
scrollable_table = f"""
<div style="height: 400px; width: 100%; overflow-x: auto; overflow-y: auto;">
    {html_table}
</div>
"""
# Display the scrollable table
display(HTML(scrollable_table))

:::

</br>

The global mean maximum row:


In [ ]:
#| echo: false
#| code-fold: false

################
dt_valid_sub = dt_valid[list(dt_valid.columns[dt_valid.columns.str.startswith('mAP_valid')])]

# Apply the function row-wise
dt_valid_sub['average_map'] = dt_valid_sub.apply(np.mean, axis=1)
dt_valid_sub.sort_values(by='average_map', ascending=False, inplace=True)

#maximum mean value
dt_valid_sub['average_map'].max()



#########


# extract max value row.

max_index = dt_valid_sub['average_map'].idxmax()
max_row = dt_valid_sub.loc[max_index]
#print(max_row)
dt_valid_sub_mw = dt_valid_sub.copy()

In [ ]:
#| echo: false
#| code-fold: false
dd = dt_valid_sub.head(10)

html_table = dd.to_html(index=True)

# Wrap in a scrollable div
scrollable_table = f"""
<div style="height: 400px; width: 100%; overflow-x: auto; overflow-y: auto;">
    {html_table}
</div>
"""
# Display the scrollable table
display(HTML(scrollable_table))

#### Model performance on validation data (Mann-Whitney U Test)

##### Statistical Significance:


Using the global mean maximum row as the reference, we perform the right-tailed t-test to identify significant hyperparameters.


In [ ]:
pvalues_mw = []
pvalues_real_mw = []

for _,index in enumerate(dt_valid_sub.index):

  _, p_value_mw = stats.mannwhitneyu(max_row[:-1], dt_valid_sub.loc[index][:-1], alternative='greater')#, method = 'asymptotic')
  pvalues_real_mw.append(p_value_mw)
  pvalues_mw.append((p_value_mw >= 0.05)*1)
  

dt_valid_sub_mw['pvalues_mw'] = pvalues_real_mw
dt_valid_sub_mw['sig_mw'] = pvalues_mw

dt_valid_sub_mw1 = dt_valid_sub_mw.query('sig_mw == 1')


max_indexx = dt_valid_sub_mw1.iloc[:, :-3].mean().idxmax()

#print(sum(pvalues),max_indexx)

In [ ]:
#| echo: false
#| code-fold: false

dt_mw = dt_valid.loc[dt_valid_sub_mw1.index].sort_values(by = ['input_size', 'output_size', 'learning_rate', 'batch_size', 'alpha', 'margin', 'l1_reg', 'l2_reg'])
html_table = dt_mw.to_html(index=True)


# Wrap in a scrollable div
scrollable_table = f"""
<div style="height: 400px; width: 100%; overflow-x: auto; overflow-y: auto;">
    {html_table}
</div>
"""
# Display the scrollable table
display(HTML(scrollable_table))

print('Size of the All data: ',dt_valid_sub_mw.shape)

print('Size of the Sig data: ',dt_mw.shape)

In [ ]:
####################################################
# Use the hyperparameters to the test data.
####################################################
optimal_params_mw1 = dt_valid.loc[dt_valid_sub_mw1.index][['input_size', 'output_size', 'learning_rate', 'batch_size', 'alpha', 'margin', 'l1_reg', 'l2_reg']]
optimal_params_mw = optimal_params_mw1.reset_index()
optimal_params_mw.drop(['index'], axis=1, inplace = True)

#rotation invariance test data with optimal parameters
test_data_mw1 = pd.merge(optimal_params_mw, dt_test, on=['input_size', 'output_size', 'learning_rate', 'batch_size', 'alpha', 'margin', 'l1_reg', 'l2_reg'])
test_data_1_sub_mw = test_data_mw1[list(test_data_mw1.columns[test_data_mw1.columns.str.startswith('mAP_test')])]

Equivalent test data


In [ ]:
#| echo: false
#| code-fold: false
test_data_mw1 = test_data_mw1.sort_values(by = ['input_size', 'output_size', 'learning_rate', 'batch_size', 'alpha', 'margin', 'l1_reg', 'l2_reg'])
html_table = test_data_mw1.to_html(index=True)

# Wrap in a scrollable div
scrollable_table = f"""
<div style="height: 400px; width: 100%; overflow-x: auto; overflow-y: auto;">
    {html_table}
</div>
"""
# Display the scrollable table
display(HTML(scrollable_table))
print('Size of the test data: ',test_data_mw1.shape)

::: {.panel-tabset}

##### Model performance on valid data 

Average & Std Deviation of the Significant rows:


In [ ]:
#| echo: false
#| code-fold: false
res = dt_valid_sub_mw1.iloc[:,:-3].describe().iloc[1:3]

res_valid_mwu_24 = res.T.sort_values(by=['mean'], ascending= False)

html_table = res_valid_mwu_24.to_html(index=True)

# Wrap in a scrollable div
scrollable_table = f"""
<div style="height: 400px; width: 100%; overflow-x: auto; overflow-y: auto;">
    {html_table}
</div>
"""
# Display the scrollable table
display(HTML(scrollable_table))


##### Model performance on Test data 
</br>
We then apply the hyperparameters to the test set and average the results.


In [ ]:
####################################################
# Use the hyperparameters to the test data.
####################################################
optimal_params = dt_valid.loc[dt_valid_sub_mw1.index][['input_size', 'output_size', 'learning_rate', 'batch_size', 'alpha', 'margin', 'l1_reg', 'l2_reg']].reset_index()
optimal_params.drop(['index'], axis=1, inplace = True)

#rotation invariance test data with optimal parameters
test_data_1 = pd.merge(optimal_params, dt_test, how='left')
test_data_1_sub = test_data_1[list(test_data_1.columns[test_data_1.columns.str.startswith('mAP_test')])]
test_data_1['average_map'] = test_data_1_sub.apply(np.mean, axis=1)

test_data_1.sort_values(by=['average_map'], ascending= False,inplace=True)

In [ ]:
#| echo: false
#| code-fold: false
res = test_data_1.describe().iloc[1:3][list(test_data_1.columns[test_data_1.columns.str.startswith('mAP_test')]) + ['average_map']]

res_test_mwu_24 = res.T.sort_values(by=['mean'], ascending= False)

html_table = res_test_mwu_24.to_html(index=True)

# Wrap in a scrollable div
scrollable_table = f"""
<div style="height: 400px; width: 100%; overflow-x: auto; overflow-y: auto;">
    {html_table}
</div>
"""
# Display the scrollable table
display(HTML(scrollable_table))

:::

Summary:


In [ ]:
#| layout-ncol: 2
#| 
def extract_number(text):
    if isinstance(text, str):
        matches = re.findall(r'\d+', text)
        return int(matches[0]) if matches else 1
    return 1


res_valid_mwu_24['id'] = res_valid_mwu_24.index.to_series().apply(extract_number)
res_test_mwu_24['id'] = res_test_mwu_24.index.to_series().apply(extract_number)

res_comb = pd.concat([res_valid_mwu_24,res_test_mwu_24])
index_series = res_comb.index.to_series()

res_comb['type'] = np.where(
    index_series.str.contains('valid', case=False), 'valid',
    np.where(index_series.str.contains('test', case=False), 'test', 'unknown')
)
res_comb = res_comb.query("type !='unknown'")

res_comb =  res_comb.reset_index(drop=True)


res_test_mwu_24 = res_test_mwu_24.sort_values(by=['id']).reset_index().query("index !='average_map'")


res_valid_mwu_24 =  res_valid_mwu_24.reset_index(drop=True)
res_valid_mwu_24.columns = ['valid_score', 'std', 'id']
res_valid_mwu_24 = res_valid_mwu_24[['valid_score', 'id']]

res_test_mwu_24.columns = ['label2', 'test_score', 'std', 'id']
res_test_mwu_24 = res_test_mwu_24[['test_score', 'id']]


result_merged_mwu_24 = pd.merge(res_valid_mwu_24,res_test_mwu_24,on='id')


result_merged_mwu_24.sort_values(by=['valid_score'], ascending=False,inplace=True)

result_merged_mwu_24 = result_merged_mwu_24[['id','valid_score','test_score']]

html_table = result_merged_mwu_24.to_html(index=True)

# Wrap in a scrollable div
scrollable_table = f"""
<div style="height: 500px; width: 100%; overflow-x: auto; overflow-y: auto;">
    {html_table}
</div>
"""
# Display the scrollable table
display(HTML(scrollable_table))

##  Bit size 16

::: {.panel-tabset}

#### Validation Data mAP Results Preview:


In [ ]:
#%%

dt_valid  = pd.read_csv(f"final paper results/merged_validation_runs_wide_format_abs_24082024_{layer_vsn}.csv")


dt_test  = pd.read_csv(f"final paper results/merged_test_runs_wide_format_abs_24082024_{layer_vsn}.csv")



output_size = 16

query = (
    "output_size == @output_size "
)

dt_valid = dt_valid.query(query)

dt_test = dt_test.query(query)

In [ ]:
#| echo: false
#| code-fold: false
#| 
html_table = dt_valid.to_html(index=True)

# Wrap in a scrollable div
scrollable_table = f"""
<div style="height: 400px; width: 100%; overflow-x: auto; overflow-y: auto;">
    {html_table}
</div>
"""
# Display the scrollable table
display(HTML(scrollable_table))

</br>

#### Test Data mAP Results Preview:


In [ ]:
#| echo: false
#| code-fold: false
#| 
html_table = dt_test.to_html(index=True)

# Wrap in a scrollable div
scrollable_table = f"""
<div style="height: 400px; width: 100%; overflow-x: auto; overflow-y: auto;">
    {html_table}
</div>
"""
# Display the scrollable table
display(HTML(scrollable_table))

:::

</br>

The global mean maximum row:


In [ ]:
#| echo: false
#| code-fold: false

################
dt_valid_sub = dt_valid[list(dt_valid.columns[dt_valid.columns.str.startswith('mAP_valid')])]

# Apply the function row-wise
dt_valid_sub['average_map'] = dt_valid_sub.apply(np.mean, axis=1)
dt_valid_sub.sort_values(by='average_map', ascending=False, inplace=True)

#maximum mean value
dt_valid_sub['average_map'].max()



#########


# extract max value row.

max_index = dt_valid_sub['average_map'].idxmax()
max_row = dt_valid_sub.loc[max_index]
#print(max_row)
dt_valid_sub_mw = dt_valid_sub.copy()

In [ ]:
#| echo: false
#| code-fold: false
dd = dt_valid_sub.head(10)

html_table = dd.to_html(index=True)

# Wrap in a scrollable div
scrollable_table = f"""
<div style="height: 400px; width: 100%; overflow-x: auto; overflow-y: auto;">
    {html_table}
</div>
"""
# Display the scrollable table
display(HTML(scrollable_table))

#### Model performance on validation data (Mann-Whitney U Test)

##### Statistical Significance:


Using the global mean maximum row as the reference, we perform the right-tailed t-test to identify significant hyperparameters.


In [ ]:
pvalues_mw = []
pvalues_real_mw = []

for _,index in enumerate(dt_valid_sub.index):

  _, p_value_mw = stats.mannwhitneyu(max_row[:-1], dt_valid_sub.loc[index][:-1], alternative='greater')#, method = 'asymptotic')
  pvalues_real_mw.append(p_value_mw)
  pvalues_mw.append((p_value_mw >= 0.05)*1)
  

dt_valid_sub_mw['pvalues_mw'] = pvalues_real_mw
dt_valid_sub_mw['sig_mw'] = pvalues_mw

dt_valid_sub_mw1 = dt_valid_sub_mw.query('sig_mw == 1')


max_indexx = dt_valid_sub_mw1.iloc[:, :-3].mean().idxmax()

#print(sum(pvalues),max_indexx)

In [ ]:
#| echo: false
#| code-fold: false

dt_mw = dt_valid.loc[dt_valid_sub_mw1.index].sort_values(by = ['input_size', 'output_size', 'learning_rate', 'batch_size', 'alpha', 'margin', 'l1_reg', 'l2_reg'])
html_table = dt_mw.to_html(index=True)


# Wrap in a scrollable div
scrollable_table = f"""
<div style="height: 400px; width: 100%; overflow-x: auto; overflow-y: auto;">
    {html_table}
</div>
"""
# Display the scrollable table
display(HTML(scrollable_table))

print('Size of the All data: ',dt_valid_sub_mw.shape)

print('Size of the Sig data: ',dt_mw.shape)

In [ ]:
####################################################
# Use the hyperparameters to the test data.
####################################################
optimal_params_mw1 = dt_valid.loc[dt_valid_sub_mw1.index][['input_size', 'output_size', 'learning_rate', 'batch_size', 'alpha', 'margin', 'l1_reg', 'l2_reg']]
optimal_params_mw = optimal_params_mw1.reset_index()
optimal_params_mw.drop(['index'], axis=1, inplace = True)

#rotation invariance test data with optimal parameters
test_data_mw1 = pd.merge(optimal_params_mw, dt_test, on=['input_size', 'output_size', 'learning_rate', 'batch_size', 'alpha', 'margin', 'l1_reg', 'l2_reg'])
test_data_1_sub_mw = test_data_mw1[list(test_data_mw1.columns[test_data_mw1.columns.str.startswith('mAP_test')])]

Equivalent test data


In [ ]:
#| echo: false
#| code-fold: false
test_data_mw1 = test_data_mw1.sort_values(by = ['input_size', 'output_size', 'learning_rate', 'batch_size', 'alpha', 'margin', 'l1_reg', 'l2_reg'])
html_table = test_data_mw1.to_html(index=True)

# Wrap in a scrollable div
scrollable_table = f"""
<div style="height: 400px; width: 100%; overflow-x: auto; overflow-y: auto;">
    {html_table}
</div>
"""
# Display the scrollable table
display(HTML(scrollable_table))
print('Size of the test data: ',test_data_mw1.shape)

::: {.panel-tabset}

##### Model performance on valid data 

Average & Std Deviation of the Significant rows:


In [ ]:
#| echo: false
#| code-fold: false
res = dt_valid_sub_mw1.iloc[:,:-3].describe().iloc[1:3]

res_valid_mwu_16 = res.T.sort_values(by=['mean'], ascending= False)

html_table = res_valid_mwu_16.to_html(index=True)

# Wrap in a scrollable div
scrollable_table = f"""
<div style="height: 400px; width: 100%; overflow-x: auto; overflow-y: auto;">
    {html_table}
</div>
"""
# Display the scrollable table
display(HTML(scrollable_table))


##### Model performance on Test data 
</br>
We then apply the hyperparameters to the test set and average the results.


In [ ]:
####################################################
# Use the hyperparameters to the test data.
####################################################
optimal_params = dt_valid.loc[dt_valid_sub_mw1.index][['input_size', 'output_size', 'learning_rate', 'batch_size', 'alpha', 'margin', 'l1_reg', 'l2_reg']].reset_index()
optimal_params.drop(['index'], axis=1, inplace = True)

#rotation invariance test data with optimal parameters
test_data_1 = pd.merge(optimal_params, dt_test, how='left')
test_data_1_sub = test_data_1[list(test_data_1.columns[test_data_1.columns.str.startswith('mAP_test')])]
test_data_1['average_map'] = test_data_1_sub.apply(np.mean, axis=1)

test_data_1.sort_values(by=['average_map'], ascending= False,inplace=True)

In [ ]:
#| echo: false
#| code-fold: false
res = test_data_1.describe().iloc[1:3][list(test_data_1.columns[test_data_1.columns.str.startswith('mAP_test')]) + ['average_map']]

res_test_mwu_16 = res.T.sort_values(by=['mean'], ascending= False)

html_table = res_test_mwu_16.to_html(index=True)

# Wrap in a scrollable div
scrollable_table = f"""
<div style="height: 400px; width: 100%; overflow-x: auto; overflow-y: auto;">
    {html_table}
</div>
"""
# Display the scrollable table
display(HTML(scrollable_table))

:::

Summary:


In [ ]:
#| layout-ncol: 2
#| 
def extract_number(text):
    if isinstance(text, str):
        matches = re.findall(r'\d+', text)
        return int(matches[0]) if matches else 1
    return 1


res_valid_mwu_16['id'] = res_valid_mwu_16.index.to_series().apply(extract_number)
res_test_mwu_16['id'] = res_test_mwu_16.index.to_series().apply(extract_number)

res_comb = pd.concat([res_valid_mwu_16,res_test_mwu_16])
index_series = res_comb.index.to_series()

res_comb['type'] = np.where(
    index_series.str.contains('valid', case=False), 'valid',
    np.where(index_series.str.contains('test', case=False), 'test', 'unknown')
)
res_comb = res_comb.query("type !='unknown'")

res_comb =  res_comb.reset_index(drop=True)


res_test_mwu_16 = res_test_mwu_16.sort_values(by=['id']).reset_index().query("index !='average_map'")


res_valid_mwu_16 =  res_valid_mwu_16.reset_index(drop=True)
res_valid_mwu_16.columns = ['valid_score', 'std', 'id']
res_valid_mwu_16 = res_valid_mwu_16[['valid_score', 'id']]

res_test_mwu_16.columns = ['label2', 'test_score', 'std', 'id']
res_test_mwu_16 = res_test_mwu_16[['test_score', 'id']]


result_merged_mwu_16 = pd.merge(res_valid_mwu_16,res_test_mwu_16,on='id')


result_merged_mwu_16.sort_values(by=['valid_score'], ascending=False,inplace=True)

result_merged_mwu_16 = result_merged_mwu_16[['id','valid_score','test_score']]

html_table = result_merged_mwu_16.to_html(index=True)

# Wrap in a scrollable div
scrollable_table = f"""
<div style="height: 500px; width: 100%; overflow-x: auto; overflow-y: auto;">
    {html_table}
</div>
"""
# Display the scrollable table
display(HTML(scrollable_table))

##  Bit size 40

::: {.panel-tabset}

#### Validation Data mAP Results Preview:


In [ ]:
#%%



dt_valid  = pd.read_csv(f"final paper results/merged_validation_runs_wide_format_abs_24082024_{layer_vsn}.csv")


dt_test  = pd.read_csv(f"final paper results/merged_test_runs_wide_format_abs_24082024_{layer_vsn}.csv")



output_size = 40

query = (
    "output_size == @output_size "
)

dt_valid = dt_valid.query(query)

dt_test = dt_test.query(query)

In [ ]:
#| echo: false
#| code-fold: false
#| 
html_table = dt_valid.to_html(index=True)

# Wrap in a scrollable div
scrollable_table = f"""
<div style="height: 400px; width: 100%; overflow-x: auto; overflow-y: auto;">
    {html_table}
</div>
"""
# Display the scrollable table
display(HTML(scrollable_table))

</br>

#### Test Data mAP Results Preview:


In [ ]:
#| echo: false
#| code-fold: false
#| 
html_table = dt_test.to_html(index=True)

# Wrap in a scrollable div
scrollable_table = f"""
<div style="height: 400px; width: 100%; overflow-x: auto; overflow-y: auto;">
    {html_table}
</div>
"""
# Display the scrollable table
display(HTML(scrollable_table))

:::

</br>

The global mean maximum row:


In [ ]:
#| echo: false
#| code-fold: false

################
dt_valid_sub = dt_valid[list(dt_valid.columns[dt_valid.columns.str.startswith('mAP_valid')])]

# Apply the function row-wise
dt_valid_sub['average_map'] = dt_valid_sub.apply(np.mean, axis=1)
dt_valid_sub.sort_values(by='average_map', ascending=False, inplace=True)

#maximum mean value
dt_valid_sub['average_map'].max()



#########


# extract max value row.

max_index = dt_valid_sub['average_map'].idxmax()
max_row = dt_valid_sub.loc[max_index]
#print(max_row)
dt_valid_sub_mw = dt_valid_sub.copy()

In [ ]:
#| echo: false
#| code-fold: false
dd = dt_valid_sub.head(10)

html_table = dd.to_html(index=True)

# Wrap in a scrollable div
scrollable_table = f"""
<div style="height: 400px; width: 100%; overflow-x: auto; overflow-y: auto;">
    {html_table}
</div>
"""
# Display the scrollable table
display(HTML(scrollable_table))

#### Model performance on validation data (Mann-Whitney U Test)

##### Statistical Significance:


Using the global mean maximum row as the reference, we perform the right-tailed t-test to identify significant hyperparameters.


In [ ]:
pvalues_mw = []
pvalues_real_mw = []

for _,index in enumerate(dt_valid_sub.index):

  _, p_value_mw = stats.mannwhitneyu(max_row[:-1], dt_valid_sub.loc[index][:-1], alternative='greater')#, method = 'asymptotic')
  pvalues_real_mw.append(p_value_mw)
  pvalues_mw.append((p_value_mw >= 0.05)*1)
  

dt_valid_sub_mw['pvalues_mw'] = pvalues_real_mw
dt_valid_sub_mw['sig_mw'] = pvalues_mw

dt_valid_sub_mw1 = dt_valid_sub_mw.query('sig_mw == 1')


max_indexx = dt_valid_sub_mw1.iloc[:, :-3].mean().idxmax()

#print(sum(pvalues),max_indexx)

In [ ]:
#| echo: false
#| code-fold: false

dt_mw = dt_valid.loc[dt_valid_sub_mw1.index].sort_values(by = ['input_size', 'output_size', 'learning_rate', 'batch_size', 'alpha', 'margin', 'l1_reg', 'l2_reg'])
html_table = dt_mw.to_html(index=True)


# Wrap in a scrollable div
scrollable_table = f"""
<div style="height: 400px; width: 100%; overflow-x: auto; overflow-y: auto;">
    {html_table}
</div>
"""
# Display the scrollable table
display(HTML(scrollable_table))

print('Size of the All data: ',dt_valid_sub_mw.shape)

print('Size of the Sig data: ',dt_mw.shape)

In [ ]:
####################################################
# Use the hyperparameters to the test data.
####################################################
optimal_params_mw1 = dt_valid.loc[dt_valid_sub_mw1.index][['input_size', 'output_size', 'learning_rate', 'batch_size', 'alpha', 'margin', 'l1_reg', 'l2_reg']]
optimal_params_mw = optimal_params_mw1.reset_index()
optimal_params_mw.drop(['index'], axis=1, inplace = True)

#rotation invariance test data with optimal parameters
test_data_mw1 = pd.merge(optimal_params_mw, dt_test, on=['input_size', 'output_size', 'learning_rate', 'batch_size', 'alpha', 'margin', 'l1_reg', 'l2_reg'])
test_data_1_sub_mw = test_data_mw1[list(test_data_mw1.columns[test_data_mw1.columns.str.startswith('mAP_test')])]

Equivalent test data


In [ ]:
#| echo: false
#| code-fold: false
test_data_mw1 = test_data_mw1.sort_values(by = ['input_size', 'output_size', 'learning_rate', 'batch_size', 'alpha', 'margin', 'l1_reg', 'l2_reg'])
html_table = test_data_mw1.to_html(index=True)

# Wrap in a scrollable div
scrollable_table = f"""
<div style="height: 400px; width: 100%; overflow-x: auto; overflow-y: auto;">
    {html_table}
</div>
"""
# Display the scrollable table
display(HTML(scrollable_table))
print('Size of the test data: ',test_data_mw1.shape)

::: {.panel-tabset}

##### Model performance on valid data 

Average & Std Deviation of the Significant rows:


In [ ]:
#| echo: false
#| code-fold: false
res = dt_valid_sub_mw1.iloc[:,:-3].describe().iloc[1:3]

res_valid_mwu_40 = res.T.sort_values(by=['mean'], ascending= False)

html_table = res_valid_mwu_40.to_html(index=True)

# Wrap in a scrollable div
scrollable_table = f"""
<div style="height: 400px; width: 100%; overflow-x: auto; overflow-y: auto;">
    {html_table}
</div>
"""
# Display the scrollable table
display(HTML(scrollable_table))


##### Model performance on Test data 
</br>
We then apply the hyperparameters to the test set and average the results.


In [ ]:
####################################################
# Use the hyperparameters to the test data.
####################################################
optimal_params = dt_valid.loc[dt_valid_sub_mw1.index][['input_size', 'output_size', 'learning_rate', 'batch_size', 'alpha', 'margin', 'l1_reg', 'l2_reg']].reset_index()
optimal_params.drop(['index'], axis=1, inplace = True)

optimal_params.to_csv('optimal_params_40bit.csv',index=False)
#rotation invariance test data with optimal parameters
test_data_1 = pd.merge(optimal_params, dt_test, how='left')
test_data_1_sub = test_data_1[list(test_data_1.columns[test_data_1.columns.str.startswith('mAP_test')])]
test_data_1['average_map'] = test_data_1_sub.apply(np.mean, axis=1)

test_data_1.sort_values(by=['average_map'], ascending= False,inplace=True)

In [ ]:
#| echo: false
#| code-fold: false
res = test_data_1.describe().iloc[1:3][list(test_data_1.columns[test_data_1.columns.str.startswith('mAP_test')]) + ['average_map']]

res_test_mwu_40 = res.T.sort_values(by=['mean'], ascending= False)

html_table = res_test_mwu_40.to_html(index=True)

# Wrap in a scrollable div
scrollable_table = f"""
<div style="height: 400px; width: 100%; overflow-x: auto; overflow-y: auto;">
    {html_table}
</div>
"""
# Display the scrollable table
display(HTML(scrollable_table))

:::


In [ ]:
def extract_number(text):
    if isinstance(text, str):
        matches = re.findall(r'\d+', text)
        return int(matches[0]) if matches else 1
    return 1

res_valid_mwu_40['id'] = res_valid_mwu_40.index.to_series().apply(extract_number)
res_test_mwu_40['id'] = res_test_mwu_40.index.to_series().apply(extract_number)

res_comb = pd.concat([res_valid_mwu_40,res_test_mwu_40])
index_series = res_comb.index.to_series()

res_comb['type'] = np.where(
    index_series.str.contains('valid', case=False), 'valid',
    np.where(index_series.str.contains('test', case=False), 'test', 'unknown')
)
res_comb = res_comb.query("type !='unknown'")

res_comb =  res_comb.reset_index(drop=True)


res_test_mwu_40 = res_test_mwu_40.sort_values(by=['id']).reset_index().query("index !='average_map'")


res_valid_mwu_40 =  res_valid_mwu_40.reset_index(drop=True)
res_valid_mwu_40.columns = ['valid_score', 'std', 'id']
res_valid_mwu_40 = res_valid_mwu_40[['valid_score', 'id']]

res_test_mwu_40.columns = ['label2', 'test_score', 'std', 'id']
res_test_mwu_40 = res_test_mwu_40[['test_score', 'id']]


result_merged_mwu_40 = pd.merge(res_valid_mwu_40,res_test_mwu_40,on='id')


result_merged_mwu_40.sort_values(by=['valid_score'], ascending=False,inplace=True)

result_merged_mwu_40 = result_merged_mwu_40[['id','valid_score','test_score']]

html_table = result_merged_mwu_40.to_html(index=True)

# Wrap in a scrollable div
scrollable_table = f"""
<div style="height: 500px; width: 100%; overflow-x: auto; overflow-y: auto;">
    {html_table}
</div>
"""
# Display the scrollable table
display(HTML(scrollable_table))

##  Bit size 56

::: {.panel-tabset}

#### Validation Data mAP Results Preview:


In [ ]:
#%%


dt_valid  = pd.read_csv(f"final paper results/merged_validation_runs_wide_format_abs_24082024_{layer_vsn}.csv")


dt_test  = pd.read_csv(f"final paper results/merged_test_runs_wide_format_abs_24082024_{layer_vsn}.csv")



output_size = 56

query = (
    "output_size == @output_size "
)

dt_valid = dt_valid.query(query)

dt_test = dt_test.query(query)

In [ ]:
#| echo: false
#| code-fold: false
#| 
html_table = dt_valid.to_html(index=True)

# Wrap in a scrollable div
scrollable_table = f"""
<div style="height: 400px; width: 100%; overflow-x: auto; overflow-y: auto;">
    {html_table}
</div>
"""
# Display the scrollable table
display(HTML(scrollable_table))

</br>

#### Test Data mAP Results Preview:


In [ ]:
#| echo: false
#| code-fold: false
#| 
html_table = dt_test.to_html(index=True)

# Wrap in a scrollable div
scrollable_table = f"""
<div style="height: 400px; width: 100%; overflow-x: auto; overflow-y: auto;">
    {html_table}
</div>
"""
# Display the scrollable table
display(HTML(scrollable_table))

:::

</br>

The global mean maximum row:


In [ ]:
#| echo: false
#| code-fold: false

################
dt_valid_sub = dt_valid[list(dt_valid.columns[dt_valid.columns.str.startswith('mAP_valid')])]

# Apply the function row-wise
dt_valid_sub['average_map'] = dt_valid_sub.apply(np.mean, axis=1)
dt_valid_sub.sort_values(by='average_map', ascending=False, inplace=True)

#maximum mean value
dt_valid_sub['average_map'].max()



#########


# extract max value row.

max_index = dt_valid_sub['average_map'].idxmax()
max_row = dt_valid_sub.loc[max_index]
#print(max_row)
dt_valid_sub_mw = dt_valid_sub.copy()

In [ ]:
#| echo: false
#| code-fold: false
dd = dt_valid_sub.head(10)

html_table = dd.to_html(index=True)

# Wrap in a scrollable div
scrollable_table = f"""
<div style="height: 400px; width: 100%; overflow-x: auto; overflow-y: auto;">
    {html_table}
</div>
"""
# Display the scrollable table
display(HTML(scrollable_table))

#### Model performance on validation data (Mann-Whitney U Test)

##### Statistical Significance:


Using the global mean maximum row as the reference, we perform the right-tailed t-test to identify significant hyperparameters.


In [ ]:
pvalues_mw = []
pvalues_real_mw = []

for _,index in enumerate(dt_valid_sub.index):

  _, p_value_mw = stats.mannwhitneyu(max_row[:-1], dt_valid_sub.loc[index][:-1], alternative='greater')#, method = 'asymptotic')
  pvalues_real_mw.append(p_value_mw)
  pvalues_mw.append((p_value_mw >= 0.05)*1)
  

dt_valid_sub_mw['pvalues_mw'] = pvalues_real_mw
dt_valid_sub_mw['sig_mw'] = pvalues_mw

dt_valid_sub_mw1 = dt_valid_sub_mw.query('sig_mw == 1')


max_indexx = dt_valid_sub_mw1.iloc[:, :-3].mean().idxmax()

#print(sum(pvalues),max_indexx)

In [ ]:
#| echo: false
#| code-fold: false

dt_mw = dt_valid.loc[dt_valid_sub_mw1.index].sort_values(by = ['input_size', 'output_size', 'learning_rate', 'batch_size', 'alpha', 'margin', 'l1_reg', 'l2_reg'])
html_table = dt_mw.to_html(index=True)


# Wrap in a scrollable div
scrollable_table = f"""
<div style="height: 400px; width: 100%; overflow-x: auto; overflow-y: auto;">
    {html_table}
</div>
"""
# Display the scrollable table
display(HTML(scrollable_table))

print('Size of the All data: ',dt_valid_sub_mw.shape)

print('Size of the Sig data: ',dt_mw.shape)

In [ ]:
####################################################
# Use the hyperparameters to the test data.
####################################################
optimal_params_mw1 = dt_valid.loc[dt_valid_sub_mw1.index][['input_size', 'output_size', 'learning_rate', 'batch_size', 'alpha', 'margin', 'l1_reg', 'l2_reg']]
optimal_params_mw = optimal_params_mw1.reset_index()
optimal_params_mw.drop(['index'], axis=1, inplace = True)

#rotation invariance test data with optimal parameters
test_data_mw1 = pd.merge(optimal_params_mw, dt_test, on=['input_size', 'output_size', 'learning_rate', 'batch_size', 'alpha', 'margin', 'l1_reg', 'l2_reg'])
test_data_1_sub_mw = test_data_mw1[list(test_data_mw1.columns[test_data_mw1.columns.str.startswith('mAP_test')])]

Equivalent test data


In [ ]:
#| echo: false
#| code-fold: false
test_data_mw1 = test_data_mw1.sort_values(by = ['input_size', 'output_size', 'learning_rate', 'batch_size', 'alpha', 'margin', 'l1_reg', 'l2_reg'])
html_table = test_data_mw1.to_html(index=True)

# Wrap in a scrollable div
scrollable_table = f"""
<div style="height: 400px; width: 100%; overflow-x: auto; overflow-y: auto;">
    {html_table}
</div>
"""
# Display the scrollable table
display(HTML(scrollable_table))
print('Size of the test data: ',test_data_mw1.shape)

::: {.panel-tabset}

##### Model performance on valid data 

Average & Std Deviation of the Significant rows:


In [ ]:
#| echo: false
#| code-fold: false
res = dt_valid_sub_mw1.iloc[:,:-3].describe().iloc[1:3]

res_valid_mwu_56 = res.T.sort_values(by=['mean'], ascending= False)

html_table = res_valid_mwu_56.to_html(index=True)

# Wrap in a scrollable div
scrollable_table = f"""
<div style="height: 400px; width: 100%; overflow-x: auto; overflow-y: auto;">
    {html_table}
</div>
"""
# Display the scrollable table
display(HTML(scrollable_table))


##### Model performance on Test data 
</br>
We then apply the hyperparameters to the test set and average the results.


In [ ]:
####################################################
# Use the hyperparameters to the test data.
####################################################
optimal_params = dt_valid.loc[dt_valid_sub_mw1.index][['input_size', 'output_size', 'learning_rate', 'batch_size', 'alpha', 'margin', 'l1_reg', 'l2_reg']].reset_index()
optimal_params.drop(['index'], axis=1, inplace = True)

optimal_params.to_csv('optimal_params_56bit.csv',index=False)
#rotation invariance test data with optimal parameters
test_data_1 = pd.merge(optimal_params, dt_test, how='left')
test_data_1_sub = test_data_1[list(test_data_1.columns[test_data_1.columns.str.startswith('mAP_test')])]
test_data_1['average_map'] = test_data_1_sub.apply(np.mean, axis=1)

test_data_1.sort_values(by=['average_map'], ascending= False,inplace=True)

In [ ]:
#| echo: false
#| code-fold: false
res = test_data_1.describe().iloc[1:3][list(test_data_1.columns[test_data_1.columns.str.startswith('mAP_test')]) + ['average_map']]

res_test_mwu_56 = res.T.sort_values(by=['mean'], ascending= False)

html_table = res_test_mwu_56.to_html(index=True)

# Wrap in a scrollable div
scrollable_table = f"""
<div style="height: 400px; width: 100%; overflow-x: auto; overflow-y: auto;">
    {html_table}
</div>
"""
# Display the scrollable table
display(HTML(scrollable_table))

:::


In [ ]:
def extract_number(text):
    if isinstance(text, str):
        matches = re.findall(r'\d+', text)
        return int(matches[0]) if matches else 1
    return 1

res_valid_mwu_56['id'] = res_valid_mwu_56.index.to_series().apply(extract_number)
res_test_mwu_56['id'] = res_test_mwu_56.index.to_series().apply(extract_number)

res_comb = pd.concat([res_valid_mwu_56,res_test_mwu_56])
index_series = res_comb.index.to_series()

res_comb['type'] = np.where(
    index_series.str.contains('valid', case=False), 'valid',
    np.where(index_series.str.contains('test', case=False), 'test', 'unknown')
)
res_comb = res_comb.query("type !='unknown'")

res_comb =  res_comb.reset_index(drop=True)


res_test_mwu_56 = res_test_mwu_56.sort_values(by=['id']).reset_index().query("index !='average_map'")


res_valid_mwu_56 =  res_valid_mwu_56.reset_index(drop=True)
res_valid_mwu_56.columns = ['valid_score', 'std', 'id']
res_valid_mwu_56 = res_valid_mwu_56[['valid_score', 'id']]

res_test_mwu_56.columns = ['label2', 'test_score', 'std', 'id']
res_test_mwu_56 = res_test_mwu_56[['test_score', 'id']]


result_merged_mwu_56 = pd.merge(res_valid_mwu_56,res_test_mwu_56,on='id')


result_merged_mwu_56.sort_values(by=['valid_score'], ascending=False,inplace=True)

result_merged_mwu_56 = result_merged_mwu_56[['id','valid_score','test_score']]

html_table = result_merged_mwu_56.to_html(index=True)

# Wrap in a scrollable div
scrollable_table = f"""
<div style="height: 500px; width: 100%; overflow-x: auto; overflow-y: auto;">
    {html_table}
</div>
"""
# Display the scrollable table
display(HTML(scrollable_table))

##  Bit size 64

::: {.panel-tabset}

#### Validation Data mAP Results Preview:


In [ ]:
#%%


dt_valid  = pd.read_csv(f"final paper results/merged_validation_runs_wide_format_abs_24082024_{layer_vsn}.csv")


dt_test  = pd.read_csv(f"final paper results/merged_test_runs_wide_format_abs_24082024_{layer_vsn}.csv")



output_size = 64

query = (
    "output_size == @output_size "
)

dt_valid = dt_valid.query(query)

dt_test = dt_test.query(query)

In [ ]:
#| echo: false
#| code-fold: false
#| 
html_table = dt_valid.to_html(index=True)

# Wrap in a scrollable div
scrollable_table = f"""
<div style="height: 400px; width: 100%; overflow-x: auto; overflow-y: auto;">
    {html_table}
</div>
"""
# Display the scrollable table
display(HTML(scrollable_table))

</br>

#### Test Data mAP Results Preview:


In [ ]:
#| echo: false
#| code-fold: false
#| 
html_table = dt_test.to_html(index=True)

# Wrap in a scrollable div
scrollable_table = f"""
<div style="height: 400px; width: 100%; overflow-x: auto; overflow-y: auto;">
    {html_table}
</div>
"""
# Display the scrollable table
display(HTML(scrollable_table))

:::

</br>

The global mean maximum row:


In [ ]:
#| echo: false
#| code-fold: false

################
dt_valid_sub = dt_valid[list(dt_valid.columns[dt_valid.columns.str.startswith('mAP_valid')])]

# Apply the function row-wise
dt_valid_sub['average_map'] = dt_valid_sub.apply(np.mean, axis=1)
dt_valid_sub.sort_values(by='average_map', ascending=False, inplace=True)

#maximum mean value
dt_valid_sub['average_map'].max()



#########


# extract max value row.

max_index = dt_valid_sub['average_map'].idxmax()
max_row = dt_valid_sub.loc[max_index]
#print(max_row)
dt_valid_sub_mw = dt_valid_sub.copy()

In [ ]:
#| echo: false
#| code-fold: false
dd = dt_valid_sub.head(10)

html_table = dd.to_html(index=True)

# Wrap in a scrollable div
scrollable_table = f"""
<div style="height: 400px; width: 100%; overflow-x: auto; overflow-y: auto;">
    {html_table}
</div>
"""
# Display the scrollable table
display(HTML(scrollable_table))

#### Model performance on validation data (Mann-Whitney U Test)

##### Statistical Significance:


Using the global mean maximum row as the reference, we perform the right-tailed t-test to identify significant hyperparameters.


In [ ]:
pvalues_mw = []
pvalues_real_mw = []

for _,index in enumerate(dt_valid_sub.index):

  _, p_value_mw = stats.mannwhitneyu(max_row[:-1], dt_valid_sub.loc[index][:-1], alternative='greater')#, method = 'asymptotic')
  pvalues_real_mw.append(p_value_mw)
  pvalues_mw.append((p_value_mw >= 0.05)*1)
  

dt_valid_sub_mw['pvalues_mw'] = pvalues_real_mw
dt_valid_sub_mw['sig_mw'] = pvalues_mw

dt_valid_sub_mw1 = dt_valid_sub_mw.query('sig_mw == 1')


max_indexx = dt_valid_sub_mw1.iloc[:, :-3].mean().idxmax()

#print(sum(pvalues),max_indexx)

In [ ]:
#| echo: false
#| code-fold: false

dt_mw = dt_valid.loc[dt_valid_sub_mw1.index].sort_values(by = ['input_size', 'output_size', 'learning_rate', 'batch_size', 'alpha', 'margin', 'l1_reg', 'l2_reg'])
html_table = dt_mw.to_html(index=True)


# Wrap in a scrollable div
scrollable_table = f"""
<div style="height: 400px; width: 100%; overflow-x: auto; overflow-y: auto;">
    {html_table}
</div>
"""
# Display the scrollable table
display(HTML(scrollable_table))

print('Size of the All data: ',dt_valid_sub_mw.shape)

print('Size of the Sig data: ',dt_mw.shape)

In [ ]:
####################################################
# Use the hyperparameters to the test data.
####################################################
optimal_params_mw1 = dt_valid.loc[dt_valid_sub_mw1.index][['input_size', 'output_size', 'learning_rate', 'batch_size', 'alpha', 'margin', 'l1_reg', 'l2_reg']]
optimal_params_mw = optimal_params_mw1.reset_index()
optimal_params_mw.drop(['index'], axis=1, inplace = True)

#rotation invariance test data with optimal parameters
test_data_mw1 = pd.merge(optimal_params_mw, dt_test, on=['input_size', 'output_size', 'learning_rate', 'batch_size', 'alpha', 'margin', 'l1_reg', 'l2_reg'])
test_data_1_sub_mw = test_data_mw1[list(test_data_mw1.columns[test_data_mw1.columns.str.startswith('mAP_test')])]

Equivalent test data


In [ ]:
#| echo: false
#| code-fold: false
test_data_mw1 = test_data_mw1.sort_values(by = ['input_size', 'output_size', 'learning_rate', 'batch_size', 'alpha', 'margin', 'l1_reg', 'l2_reg'])
html_table = test_data_mw1.to_html(index=True)

# Wrap in a scrollable div
scrollable_table = f"""
<div style="height: 400px; width: 100%; overflow-x: auto; overflow-y: auto;">
    {html_table}
</div>
"""
# Display the scrollable table
display(HTML(scrollable_table))
print('Size of the test data: ',test_data_mw1.shape)

::: {.panel-tabset}

##### Model performance on valid data 

Average & Std Deviation of the Significant rows:


In [ ]:
#| echo: false
#| code-fold: false
res = dt_valid_sub_mw1.iloc[:,:-3].describe().iloc[1:3]

res_valid_mwu_64 = res.T.sort_values(by=['mean'], ascending= False)

html_table = res_valid_mwu_64.to_html(index=True)

# Wrap in a scrollable div
scrollable_table = f"""
<div style="height: 400px; width: 100%; overflow-x: auto; overflow-y: auto;">
    {html_table}
</div>
"""
# Display the scrollable table
display(HTML(scrollable_table))


##### Model performance on Test data 
</br>
We then apply the hyperparameters to the test set and average the results.


In [ ]:
####################################################
# Use the hyperparameters to the test data.
####################################################
optimal_params = dt_valid.loc[dt_valid_sub_mw1.index][['input_size', 'output_size', 'learning_rate', 'batch_size', 'alpha', 'margin', 'l1_reg', 'l2_reg']].reset_index()
optimal_params.drop(['index'], axis=1, inplace = True)

optimal_params.to_csv('optimal_params_64bit.csv',index=False)
#rotation invariance test data with optimal parameters
test_data_1 = pd.merge(optimal_params, dt_test, how='left')
test_data_1_sub = test_data_1[list(test_data_1.columns[test_data_1.columns.str.startswith('mAP_test')])]
test_data_1['average_map'] = test_data_1_sub.apply(np.mean, axis=1)

test_data_1.sort_values(by=['average_map'], ascending= False,inplace=True)

In [ ]:
#| echo: false
#| code-fold: false
res = test_data_1.describe().iloc[1:3][list(test_data_1.columns[test_data_1.columns.str.startswith('mAP_test')]) + ['average_map']]

res_test_mwu_64 = res.T.sort_values(by=['mean'], ascending= False)

html_table = res_test_mwu_64.to_html(index=True)

# Wrap in a scrollable div
scrollable_table = f"""
<div style="height: 400px; width: 100%; overflow-x: auto; overflow-y: auto;">
    {html_table}
</div>
"""
# Display the scrollable table
display(HTML(scrollable_table))

:::


In [ ]:
def extract_number(text):
    if isinstance(text, str):
        matches = re.findall(r'\d+', text)
        return int(matches[0]) if matches else 1
    return 1

res_valid_mwu_64['id'] = res_valid_mwu_64.index.to_series().apply(extract_number)
res_test_mwu_64['id'] = res_test_mwu_64.index.to_series().apply(extract_number)

res_comb = pd.concat([res_valid_mwu_64,res_test_mwu_64])
index_series = res_comb.index.to_series()

res_comb['type'] = np.where(
    index_series.str.contains('valid', case=False), 'valid',
    np.where(index_series.str.contains('test', case=False), 'test', 'unknown')
)
res_comb = res_comb.query("type !='unknown'")

res_comb =  res_comb.reset_index(drop=True)


res_test_mwu_64 = res_test_mwu_64.sort_values(by=['id']).reset_index().query("index !='average_map'")


res_valid_mwu_64 =  res_valid_mwu_64.reset_index(drop=True)
res_valid_mwu_64.columns = ['valid_score', 'std', 'id']
res_valid_mwu_64 = res_valid_mwu_64[['valid_score', 'id']]

res_test_mwu_64.columns = ['label2', 'test_score', 'std', 'id']
res_test_mwu_64 = res_test_mwu_64[['test_score', 'id']]


result_merged_mwu_64 = pd.merge(res_valid_mwu_64,res_test_mwu_64,on='id')


result_merged_mwu_64.sort_values(by=['valid_score'], ascending=False,inplace=True)

result_merged_mwu_64 = result_merged_mwu_64[['id','valid_score','test_score']]

html_table = result_merged_mwu_64.to_html(index=True)

# Wrap in a scrollable div
scrollable_table = f"""
<div style="height: 500px; width: 100%; overflow-x: auto; overflow-y: auto;">
    {html_table}
</div>
"""
# Display the scrollable table
display(HTML(scrollable_table))

##  Bit size 72

::: {.panel-tabset}

#### Validation Data mAP Results Preview:


In [ ]:
#%%

dt_valid  = pd.read_csv(f"final paper results/merged_validation_runs_wide_format_abs_24082024_{layer_vsn}.csv")

dt_test  = pd.read_csv(f"final paper results/merged_test_runs_wide_format_abs_24082024_{layer_vsn}.csv")



output_size = 72

query = (
    "output_size == @output_size "
)

dt_valid = dt_valid.query(query)

dt_test = dt_test.query(query)

In [ ]:
#| echo: false
#| code-fold: false
#| 
html_table = dt_valid.to_html(index=True)

# Wrap in a scrollable div
scrollable_table = f"""
<div style="height: 400px; width: 100%; overflow-x: auto; overflow-y: auto;">
    {html_table}
</div>
"""
# Display the scrollable table
display(HTML(scrollable_table))

</br>

#### Test Data mAP Results Preview:


In [ ]:
#| echo: false
#| code-fold: false
#| 
html_table = dt_test.to_html(index=True)

# Wrap in a scrollable div
scrollable_table = f"""
<div style="height: 400px; width: 100%; overflow-x: auto; overflow-y: auto;">
    {html_table}
</div>
"""
# Display the scrollable table
display(HTML(scrollable_table))

:::

</br>

The global mean maximum row:


In [ ]:
#| echo: false
#| code-fold: false

################
dt_valid_sub = dt_valid[list(dt_valid.columns[dt_valid.columns.str.startswith('mAP_valid')])]

# Apply the function row-wise
dt_valid_sub['average_map'] = dt_valid_sub.apply(np.mean, axis=1)
dt_valid_sub.sort_values(by='average_map', ascending=False, inplace=True)

#maximum mean value
dt_valid_sub['average_map'].max()



#########


# extract max value row.

max_index = dt_valid_sub['average_map'].idxmax()
max_row = dt_valid_sub.loc[max_index]
#print(max_row)
dt_valid_sub_mw = dt_valid_sub.copy()

In [ ]:
#| echo: false
#| code-fold: false
dd = dt_valid_sub.head(10)

html_table = dd.to_html(index=True)

# Wrap in a scrollable div
scrollable_table = f"""
<div style="height: 400px; width: 100%; overflow-x: auto; overflow-y: auto;">
    {html_table}
</div>
"""
# Display the scrollable table
display(HTML(scrollable_table))

#### Model performance on validation data (Mann-Whitney U Test)

##### Statistical Significance:


Using the global mean maximum row as the reference, we perform the right-tailed t-test to identify significant hyperparameters.


In [ ]:
pvalues_mw = []
pvalues_real_mw = []

for _,index in enumerate(dt_valid_sub.index):

  _, p_value_mw = stats.mannwhitneyu(max_row[:-1], dt_valid_sub.loc[index][:-1], alternative='greater')#, method = 'asymptotic')
  pvalues_real_mw.append(p_value_mw)
  pvalues_mw.append((p_value_mw >= 0.05)*1)
  

dt_valid_sub_mw['pvalues_mw'] = pvalues_real_mw
dt_valid_sub_mw['sig_mw'] = pvalues_mw

dt_valid_sub_mw1 = dt_valid_sub_mw.query('sig_mw == 1')


max_indexx = dt_valid_sub_mw1.iloc[:, :-3].mean().idxmax()

#print(sum(pvalues),max_indexx)

In [ ]:
#| echo: false
#| code-fold: false

dt_mw = dt_valid.loc[dt_valid_sub_mw1.index].sort_values(by = ['input_size', 'output_size', 'learning_rate', 'batch_size', 'alpha', 'margin', 'l1_reg', 'l2_reg'])
html_table = dt_mw.to_html(index=True)


# Wrap in a scrollable div
scrollable_table = f"""
<div style="height: 400px; width: 100%; overflow-x: auto; overflow-y: auto;">
    {html_table}
</div>
"""
# Display the scrollable table
display(HTML(scrollable_table))

print('Size of the All data: ',dt_valid_sub_mw.shape)

print('Size of the Sig data: ',dt_mw.shape)

In [ ]:
####################################################
# Use the hyperparameters to the test data.
####################################################
optimal_params_mw1 = dt_valid.loc[dt_valid_sub_mw1.index][['input_size', 'output_size', 'learning_rate', 'batch_size', 'alpha', 'margin', 'l1_reg', 'l2_reg']]
optimal_params_mw = optimal_params_mw1.reset_index()
optimal_params_mw.drop(['index'], axis=1, inplace = True)

#rotation invariance test data with optimal parameters
test_data_mw1 = pd.merge(optimal_params_mw, dt_test, on=['input_size', 'output_size', 'learning_rate', 'batch_size', 'alpha', 'margin', 'l1_reg', 'l2_reg'])
test_data_1_sub_mw = test_data_mw1[list(test_data_mw1.columns[test_data_mw1.columns.str.startswith('mAP_test')])]

Equivalent test data


In [ ]:
#| echo: false
#| code-fold: false
test_data_mw1 = test_data_mw1.sort_values(by = ['input_size', 'output_size', 'learning_rate', 'batch_size', 'alpha', 'margin', 'l1_reg', 'l2_reg'])
html_table = test_data_mw1.to_html(index=True)

# Wrap in a scrollable div
scrollable_table = f"""
<div style="height: 400px; width: 100%; overflow-x: auto; overflow-y: auto;">
    {html_table}
</div>
"""
# Display the scrollable table
display(HTML(scrollable_table))
print('Size of the test data: ',test_data_mw1.shape)

::: {.panel-tabset}

##### Model performance on valid data 

Average & Std Deviation of the Significant rows:


In [ ]:
#| echo: false
#| code-fold: false
res = dt_valid_sub_mw1.iloc[:,:-3].describe().iloc[1:3]

res_valid_mwu_72 = res.T.sort_values(by=['mean'], ascending= False)

html_table = res_valid_mwu_72.to_html(index=True)

# Wrap in a scrollable div
scrollable_table = f"""
<div style="height: 400px; width: 100%; overflow-x: auto; overflow-y: auto;">
    {html_table}
</div>
"""
# Display the scrollable table
display(HTML(scrollable_table))


##### Model performance on Test data 
</br>
We then apply the hyperparameters to the test set and average the results.


In [ ]:
####################################################
# Use the hyperparameters to the test data.
####################################################
optimal_params = dt_valid.loc[dt_valid_sub_mw1.index][['input_size', 'output_size', 'learning_rate', 'batch_size', 'alpha', 'margin', 'l1_reg', 'l2_reg']].reset_index()
optimal_params.drop(['index'], axis=1, inplace = True)

optimal_params.to_csv('optimal_params_72bit.csv',index=False)
#rotation invariance test data with optimal parameters
test_data_1 = pd.merge(optimal_params, dt_test, how='left')
test_data_1_sub = test_data_1[list(test_data_1.columns[test_data_1.columns.str.startswith('mAP_test')])]
test_data_1['average_map'] = test_data_1_sub.apply(np.mean, axis=1)

test_data_1.sort_values(by=['average_map'], ascending= False,inplace=True)

In [ ]:
#| echo: false
#| code-fold: false
res = test_data_1.describe().iloc[1:3][list(test_data_1.columns[test_data_1.columns.str.startswith('mAP_test')]) + ['average_map']]

res_test_mwu_72 = res.T.sort_values(by=['mean'], ascending= False)

html_table = res_test_mwu_72.to_html(index=True)

# Wrap in a scrollable div
scrollable_table = f"""
<div style="height: 400px; width: 100%; overflow-x: auto; overflow-y: auto;">
    {html_table}
</div>
"""
# Display the scrollable table
display(HTML(scrollable_table))

dt = test_data_1[list(test_data_1.columns[test_data_1.columns.str.startswith('mAP_test')])]
dt['max_map'] = dt.apply(np.max, axis=1)
dt = dt.sort_values(by=['max_map'], ascending= False)

:::


In [ ]:
def extract_number(text):
    if isinstance(text, str):
        matches = re.findall(r'\d+', text)
        return int(matches[0]) if matches else 1
    return 1

res_valid_mwu_72['id'] = res_valid_mwu_72.index.to_series().apply(extract_number)
res_test_mwu_72['id'] = res_test_mwu_72.index.to_series().apply(extract_number)

res_comb = pd.concat([res_valid_mwu_72,res_test_mwu_72])
index_series = res_comb.index.to_series()

res_comb['type'] = np.where(
    index_series.str.contains('valid', case=False), 'valid',
    np.where(index_series.str.contains('test', case=False), 'test', 'unknown')
)
res_comb = res_comb.query("type !='unknown'")

res_comb =  res_comb.reset_index(drop=True)


res_test_mwu_72 = res_test_mwu_72.sort_values(by=['id']).reset_index().query("index !='average_map'")


res_valid_mwu_72 =  res_valid_mwu_72.reset_index(drop=True)
res_valid_mwu_72.columns = ['valid_score', 'std', 'id']
res_valid_mwu_72 = res_valid_mwu_72[['valid_score', 'id']]

res_test_mwu_72.columns = ['label2', 'test_score', 'std', 'id']
res_test_mwu_72 = res_test_mwu_72[['test_score', 'id']]


result_merged_mwu_72 = pd.merge(res_valid_mwu_72,res_test_mwu_72,on='id')


result_merged_mwu_72.sort_values(by=['valid_score'], ascending=False,inplace=True)

result_merged_mwu_72 = result_merged_mwu_72[['id','valid_score','test_score']]

html_table = result_merged_mwu_72.to_html(index=True)

# Wrap in a scrollable div
scrollable_table = f"""
<div style="height: 500px; width: 100%; overflow-x: auto; overflow-y: auto;">
    {html_table}
</div>
"""
# Display the scrollable table
display(HTML(scrollable_table))

:::


In [ ]:
# result_merged_mwu_8_best = result_merged_mwu_8.describe().iloc[1:2].reset_index()[['valid_score', 'test_score']]
# result_merged_mwu_8_best['bit'] = 8
result_merged_mwu_16_best = result_merged_mwu_16.describe().iloc[1:2].reset_index()[['valid_score', 'test_score']]
result_merged_mwu_16_best['bit'] = 16
result_merged_mwu_24_best = result_merged_mwu_24.describe().iloc[1:2].reset_index()[['valid_score', 'test_score']]
result_merged_mwu_24_best['bit'] = 24
result_merged_mwu_32_best = result_merged_mwu_32.describe().iloc[1:2].reset_index()[['valid_score', 'test_score']]
result_merged_mwu_32_best['bit'] = 32
result_merged_mwu_40_best = result_merged_mwu_40.describe().iloc[1:2].reset_index()[['valid_score', 'test_score']]
result_merged_mwu_40_best['bit'] = 40
result_merged_mwu_48_best = result_merged_mwu_48.describe().iloc[1:2].reset_index()[['valid_score', 'test_score']]
result_merged_mwu_48_best['bit'] = 48
result_merged_mwu_56_best = result_merged_mwu_56.describe().iloc[1:2].reset_index()[['valid_score', 'test_score']]
result_merged_mwu_56_best['bit'] = 56
result_merged_mwu_64_best = result_merged_mwu_64.describe().iloc[1:2].reset_index()[['valid_score', 'test_score']]
result_merged_mwu_64_best['bit'] = 64
result_merged_mwu_72_best = result_merged_mwu_72.describe().iloc[1:2].reset_index()[['valid_score', 'test_score']]
result_merged_mwu_72_best['bit'] = 72

result_merged_mwu_best = pd.concat([result_merged_mwu_16_best,result_merged_mwu_24_best,result_merged_mwu_32_best,result_merged_mwu_40_best,result_merged_mwu_48_best,result_merged_mwu_56_best,result_merged_mwu_64_best,result_merged_mwu_72_best])
result_merged_mwu_best = result_merged_mwu_best[['valid_score', 'test_score', 'bit']]

df_melted = pd.melt(result_merged_mwu_best, id_vars=['bit'], var_name='Metric', value_name='mAP')


# result_merged_mwu_8_std = result_merged_mwu_8.describe().iloc[2:3].reset_index()[['valid_score', 'test_score']]
# result_merged_mwu_8_std['bit'] = 8
result_merged_mwu_16_std = result_merged_mwu_16.describe().iloc[2:3].reset_index()[['valid_score', 'test_score']]
result_merged_mwu_16_std['bit'] = 16
result_merged_mwu_24_std = result_merged_mwu_24.describe().iloc[2:3].reset_index()[['valid_score', 'test_score']]
result_merged_mwu_24_std['bit'] = 24
result_merged_mwu_32_std = result_merged_mwu_32.describe().iloc[2:3].reset_index()[['valid_score', 'test_score']]
result_merged_mwu_32_std['bit'] = 32
result_merged_mwu_40_std = result_merged_mwu_40.describe().iloc[2:3].reset_index()[['valid_score', 'test_score']]
result_merged_mwu_40_std['bit'] = 40
result_merged_mwu_48_std = result_merged_mwu_48.describe().iloc[2:3].reset_index()[['valid_score', 'test_score']]
result_merged_mwu_48_std['bit'] = 48
result_merged_mwu_56_std = result_merged_mwu_56.describe().iloc[2:3].reset_index()[['valid_score', 'test_score']]
result_merged_mwu_56_std['bit'] = 56
result_merged_mwu_64_std = result_merged_mwu_64.describe().iloc[2:3].reset_index()[['valid_score', 'test_score']]
result_merged_mwu_64_std['bit'] = 64
result_merged_mwu_72_std = result_merged_mwu_72.describe().iloc[2:3].reset_index()[['valid_score', 'test_score']]
result_merged_mwu_72_std['bit'] = 72

result_merged_mwu_std = pd.concat([result_merged_mwu_16_std,result_merged_mwu_24_std,result_merged_mwu_32_std,result_merged_mwu_40_std,result_merged_mwu_48_std,result_merged_mwu_56_std,result_merged_mwu_64_std,result_merged_mwu_72_std])
result_merged_mwu_std = result_merged_mwu_std[['valid_score', 'test_score', 'bit']]
result_merged_mwu_std.columns = ['valid_score_std', 'test_score_std', 'bit']
df_melted_std = pd.melt(result_merged_mwu_std, id_vars=['bit'], var_name='Metric', value_name='mAP')
df_melted_std.to_csv('df_melted_std.csv', index=False)
df_melted.to_csv('df_melted.csv', index=False)

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd

# import matplotlib.font_manager
# matplotlib.font_manager.findSystemFonts(fontpaths=None, fontext='ttf')

sns.set_style("white")

SMALL_SIZE = 7
MEDIUM_SIZE = 7
BIGGER_SIZE = 7

plt.rc('font', size=SMALL_SIZE)          # controls default text sizes
plt.rc('axes', titlesize=SMALL_SIZE)     # fontsize of the axes title
plt.rc('axes', labelsize=MEDIUM_SIZE)    # fontsize of the x and y labels
plt.rc('xtick', labelsize=SMALL_SIZE)    # fontsize of the tick labels
plt.rc('ytick', labelsize=SMALL_SIZE)    # fontsize of the tick labels
plt.rc('legend', fontsize=SMALL_SIZE)    # legend fontsize
#plt.rc('figure', titlesize=BIGGER_SIZE)
# Data
# Create the plot
fig, ax = plt.subplots(figsize=(10/3, 3))
sns.set_style("whitegrid")
plt.legend(loc = 'upper left')
# Show the plot
plt.rc('font', family='Nimbus Roman')

df = pd.merge(result_merged_mwu_best, result_merged_mwu_std, on=['bit'])
df = df.query('bit!=8')
# Create line plot
sns.lineplot(data=df, x='bit', y='test_score',label='Test',  color='blue')
sns.lineplot(data=df, x='bit', y='valid_score',label='Valid',  color='red')
# sns.lineplot(data=df, x='bit', y='valid_score_std',  color='blue')
# sns.lineplot(data=df, x='bit', y='test_score_std',  color='red')
# Fill standard deviation envelopes for 'test_score' and 'valid_score'
plt.fill_between(df['bit'], df['test_score'] - df['test_score_std'], df['test_score'] + df['test_score_std'], color='#439CEF')
plt.fill_between(df['bit'], df['valid_score'] - df['valid_score_std'], df['valid_score'] + df['valid_score_std'], color='#ffbaba')

plt.scatter(72, df['test_score'].iloc[7], color='blue', marker='o', s=25)
plt.scatter(72, df['valid_score'].iloc[7], color='red', marker='o', s=25)

ax.set_ylabel('mAP')
ax.set_xlabel('Bit size')
ax = plt.gca()
# Set y-axis to start from a specific value
plt.ylim(85, 93)

# Set x-axis labels and ticks
x_labels = np.unique(np.array(df.bit))
plt.xticks(ticks=x_labels, labels=x_labels)
plt.grid(False)
# Show the plot
plt.tight_layout()
plt.savefig('paper images/valid_test.svg',format='svg', dpi=1200)
plt.show()

print('average performance for all the 26 discriptors in 72 bit: ')
res = result_merged_mwu_72[['valid_score','test_score']].describe()

html_table = res.to_html(index=True)

# Wrap in a scrollable div
scrollable_table = f"""
<div style="height: 500px; width: 100%; overflow-x: auto; overflow-y: auto;">
    {html_table}
</div>
"""
# Display the scrollable table
display(HTML(scrollable_table))

:::
